<a href="https://colab.research.google.com/github/deemanth-05/Mini-Project/blob/main/Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir images

In [ ]:
!pip install IPython-autotime
%load_ext autotime

In [ ]:
                    # PREPROCESSING

import os 
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from skimage.transform import resize

target = []
images = []
flat_data = []

datadir = '/content/images'
categories = ['Red','Green','Blue']

                    # LABEL ENCODING

for category in categories:
  class_num = categories.index(category)

                    # PATH

  path = os.path.join(datadir,category)
  for img in os.listdir(path):
    img_array = imread(os.path.join(path,img))
    #print(img_array.shape)
    #plt.imshow(img_array)
    
                  # RESIZE

    img_resized = resize(img_array,(150,150,3))

                  # FLATTEN
                  
    flat_data.append(img_resized.flatten())
    images.append(img_resized)
    target.append(class_num)

flat_data = np.array(flat_data)
target = np.array(target)
images = np.array(images)

In [ ]:
len(flat_data[0])

In [ ]:
target

In [ ]:
unique,count = np.unique(target,return_counts=True)
plt.bar(categories,count)

In [ ]:
                    # TRAINING AND TESTING

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(flat_data,target,test_size=0.3,random_state=100)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
param_grid = [
              {'C':[1,10,100,1000],'kernel':['linear']},
              {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']}
] 

svc = svm.SVC(probability=True)
clf = GridSearchCV(svc,param_grid)
clf.fit(x_train,y_train)

In [ ]:
y_pred = clf.predict(x_test)
y_pred

In [ ]:
y_test

In [45]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

time: 1.21 ms (started: 2021-05-22 12:30:06 +00:00)


In [ ]:
accuracy_score(y_pred,y_test)

In [48]:
confusion_matrix(y_pred,y_test)

array([[15,  0,  0],
       [ 0, 11,  0],
       [ 1,  0, 18]])

time: 10.3 ms (started: 2021-05-22 12:32:05 +00:00)


In [49]:
 print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        15
           1       1.00      1.00      1.00        11
           2       1.00      0.95      0.97        19

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45

time: 12.1 ms (started: 2021-05-22 12:32:18 +00:00)


In [ ]:
                    # SAVING THE MODEL

import pickle
pickle.dump(clf,open('img_model.p','wb'))

In [ ]:
model = pickle.load(open('img_model.p','rb'))

In [ ]:
                    # TESTING

flat_data = []
url = input('Enter Your URL')
img = imread(url)
img_resized = resize(img,(150,150,3))
flat_data.append(img_resized.flatten())
flat_data = np.array(flat_data)
print(img.shape)
plt.imshow(img_resized)
y_out = model.predict(flat_data)
y_out = categories[y_out[0]]
print(f'PREDICTED OUTPUT: {y_out}')

In [ ]:
!pip install streamlit
!pip install pyngrok==4.1.1

In [ ]:
%%writefile ImageClassifier.py
import streamlit as st
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import pickle
from PIL import Image
st.set_option('deprecation.showfileUploaderEncoding', False)
st.title('Image Classifer')
st.header('Colour Classification')
st.text('Upload the Image')

model = pickle.load(open('img_model.p','rb'))

uploaded_file = st.file_uploader("choose an image.....", type="jpg")

if uploaded_file is not None:
  img = Image.open(uploaded_file)
  st.image(img,caption='Uploaded Image')

  if st.button('PREDICT'):
    categories = ['Red','Green','Blue']
    st.header('RESULT')
    flat_data = []
    img = np.array(img)
    img_resized = resize(img,(150,150,3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)
    y_out = model.predict(flat_data)
    y_out = categories[y_out[0]]
    st.subheader(f'PREDICTED OUTPUT: {y_out}')
    q = model.predict_proba(flat_data)
    for index, item in enumerate(categories):
      st.write(f'{item} : {q[0][index]*100}%')

In [ ]:
from pyngrok import ngrok

In [ ]:
!nohup streamlit run ImageClassifier.py &
url = ngrok.connect(port='8501')
url

In [ ]:
ngrok.kill()